In [ ]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
# Downloaded this on local instead

# Character level language model!!

In [1]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [4]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115393


In [5]:
# let's look at the first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [6]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [12]:
chars[0], chars[1]  # hehe new line and space. So below, everything that's a '0' in the encoded string is a new line :P

('\n', ' ')

In [7]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: given some string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))
# "tokenize" is just encoding the input sequence (words) into integers. lol.

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [8]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([1115393]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [16]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [17]:
block_size = 8 # context length
train_data[:block_size + 1] # +1 because there are 8 possible examples coming out of this (character level examples; see the below)

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [23]:
x = train_data[:block_size]
y = train_data[1:block_size+1] # basically scroll the train data +1 right.
for t in range(block_size):
    context = x[:t+1] # btw when t=0, x[:0+1] = x[:1] = x[0] since it's exclusive
    target = y[t]
    print(f"when input is {context} the target: {target}")
x, y

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


(tensor([18, 47, 56, 57, 58,  1, 15, 47]),
 tensor([47, 56, 57, 58,  1, 15, 47, 58]))

In [28]:
torch.randint(len(data)-block_size, (batch_size,))

tensor([1082345,  243718,  750335,  905960])

In [26]:
# Introducing batch dimensions
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split: str):
	# generate a small batch of data of inputs x and targets y
	data = train_data if split == 'train' else val_data
	ix = torch.randint(len(data)-block_size, (batch_size,)) # right now, 4 numbers (batch size #)'s generated between 0 and len(data)-block_size. Random offsets in the training set
	x = torch.stack([data[i:i+block_size] for i in ix]) # get our 4 contexts and stack them
	y = torch.stack([data[i+1:i+block_size+1] for i in ix]) # get the 4 respective targets (i.e shifted one right) and stack them
	return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
	for t in range(block_size): # time dimension
		context = xb[b, :t+1]  # Remember, this gets the first *t* elements of the b row, since :x is exclusive.
		target = yb[b, t]
		print(f"when input is {context.tolist()} the target: {target}")
# 32 independent examples packed into the input xb and yb.

inputs:
torch.Size([4, 8])
tensor([[53, 59,  6,  1, 58, 56, 47, 40],
        [49, 43, 43, 54,  1, 47, 58,  1],
        [13, 52, 45, 43, 50, 53,  8,  0],
        [ 1, 39,  1, 46, 53, 59, 57, 43]])
targets:
torch.Size([4, 8])
tensor([[59,  6,  1, 58, 56, 47, 40, 59],
        [43, 43, 54,  1, 47, 58,  1, 58],
        [52, 45, 43, 50, 53,  8,  0, 26],
        [39,  1, 46, 53, 59, 57, 43,  0]])
----
when input is [53] the target: 59
when input is [53, 59] the target: 6
when input is [53, 59, 6] the target: 1
when input is [53, 59, 6, 1] the target: 58
when input is [53, 59, 6, 1, 58] the target: 56
when input is [53, 59, 6, 1, 58, 56] the target: 47
when input is [53, 59, 6, 1, 58, 56, 47] the target: 40
when input is [53, 59, 6, 1, 58, 56, 47, 40] the target: 59
when input is [49] the target: 43
when input is [49, 43] the target: 43
when input is [49, 43, 43] the target: 54
when input is [49, 43, 43, 54] the target: 1
when input is [49, 43, 43, 54, 1] the target: 47
when input is [49, 43, 

In [29]:
print(xb) # our input to the transformer

tensor([[53, 59,  6,  1, 58, 56, 47, 40],
        [49, 43, 43, 54,  1, 47, 58,  1],
        [13, 52, 45, 43, 50, 53,  8,  0],
        [ 1, 39,  1, 46, 53, 59, 57, 43]])


In [70]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C), i.e (4, 8, 65). C is  # channels, vocab size in this case. So yeah, it converts all our inputs to have a 65 embedding thingy.

        if targets == None:
            loss = None # if we're not using targets (like for generation) just have none
        else:
            B,T,C =logits.shape # Convert so we can get the expect (B,C,T) for cross entropy
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx) # ||  Returns a 3D logits because we have no target, so it just embeds the index without doing anything else.

            # we only care about last time step so grab that
            logits = logits[:, -1, :] # becomes (B, C)

            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1) || In each batch dimension, we should have a single prediction (int) for what comes next.
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1) || Add new int prediction to the already existing context.
        return idx


m = BigramLanguageModel(vocab_size)
out, loss = m(xb, yb)
print(out.shape, loss)

# For generation
@torch.no_grad() # hey I added this :o
def generate_content(model, max_tokens=100):
	idx = torch.zeros((1,1), dtype=torch.long) # A little 1,1: 1 batch size, 1 time. Zero will be how we start the generation (i.e new line)
	generated_ints = model.generate(idx, max_new_tokens=max_tokens)[0].tolist() # index into 0 to unpluck the single batch dim that exists, so we get our timesteps indices thingies.
	print(decode(generated_ints)) # convert the generated ints to text
generate_content(m)

torch.Size([256, 65]) tensor(4.7471, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [71]:
# create a PyTorch optimizer || Oh sheeeet!! Okay, so in our makemore series, we always just updated our gradients with 'stochastic gradient descent' (would be optim.SGD(...)), but adamW is a big W up. It's better. gigachad.
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [73]:
# And our training loop!! Look how fucking small it is LOL
# We always went forward pass, backward pass, update gradients. 
# So using the optimiser, this is how you do it. Bruh the loop is 5 lines of code:
batch_size = 32
for _ in range(10000):
	# forward pass
	xb, yb = get_batch('train') 
	logits, loss = m(xb, yb)
	# backward pass 
	optimizer.zero_grad(set_to_none=True)
	loss.backward()
	# gradient descent/updating gradients with adamW now :o
	optimizer.step()

print(loss.item())

2.4266703128814697


His comments are just:
```py
batch_size = 32
for steps in range(100): # increase number of steps for good results... 
    
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())
```
Optimiser step is indeed our updating :P


In [74]:
generate_content(m, max_tokens=500) # I mean... It's doing something!!


Mar is be eree t anverks m, t. whither, y ponofuthyfe br yoo wh o anse hinof ken he anu thor:
I

BUK:
SThais I tor qul s o thes t s is PSTabe'stouritavacitceif pomy indilagZSor buru hade himy are bled.
ILI pt ith ht.
THok, alle ag LAUTh LIOr ais proangre memford g fomy Seer id, aban y f ugg ue,
w; yotctomoorioryowas my cy hie,
Youeietharg wicajun
cousl fe s sopat, t:
Am-teas wnse be f CKI be itoupprsar.
Thicef s soyo hado'skince chit.

Thi'sangeff s I nd t e: wanenove s t Ineanand m,

Fonghinirm


## Okay, that's the basic bigram model!

Before we go onto letting the model see previous context more than just its immediately last point, let's move this stuff to a python script first.